In [42]:
from simple_image_download import simple_image_download as simp

# Create an instance of the Downloader class
downloader = simp.Downloader()

# Use the downloader object to perform operations
# For example, to download images:
downloader.download(keywords='Trump', limit=10)

[========================================================================] 100%


In [11]:

import cv2
import os

# Input Path
video_path = ".\\scan3.mp4"

# Output Path
output_dir = ".\\training\\aswin"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [43]:
# Create a VideoCapture object to read the input video
cap = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('total frames:',total_frames)

# Calculate the frame interval to capture for 150 images
no_samples = 5

if no_samples > total_frames :
    print('video length is too short for requested sample size')
else:
    frame_interval = total_frames // no_samples # change this number according to your needs 
    print('frame_interval:',frame_interval)

    # Set the initial frame counter to 0
    frame_counter = 0

    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        
        if not ret:
            break
        #I want to sample first 2 minutes of 5 minutes with frame interval 3 frames
        # Check if this is the frame to capture
        if frame_counter % frame_interval == 0 and frame_counter // frame_interval < no_samples:
            # Save the frame as a JPEG image
            output_path = os.path.join(output_dir, f'{frame_counter//frame_interval+1 :02}.jpg')
            cv2.imwrite(output_path, frame)
        
        # Increment the frame counter
        frame_counter += 1

    # Release the video capture object
    cap.release()
    print("\n\nDONE\n\n")


total frames: 113
frame_interval: 22


DONE




In [1]:
# Face Detection with MTCNN (Multi-Task Cascaded Convolutional Neural Network)
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # check if face was detected
    if results:
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array
    else:
        return None

# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = []
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # check if face was detected
        if face is not None:
            # store
            faces.append(face)
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = [], []
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '\\'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

# load train dataset
trainX, trainy = load_dataset('.\\training\\')
print(trainX.shape, trainy.shape)
# save arrays to one file in compressed format
savez_compressed('.\\face-detection.npz', trainX, trainy)

>loaded 63 examples for class: aswin
>loaded 108 examples for class: Trump
(171, 160, 160, 3) (171,)


In [2]:
# Face Feature Extracion using FaceNet (calculating the face embedding)
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet

# get the face embedding for one face
def get_embedding(model, face_pixels):
 # scale pixel values
 face_pixels = face_pixels.astype('float32')
 # standardize pixel values across channels (global)
 mean, std = face_pixels.mean(), face_pixels.std()
 face_pixels = (face_pixels - mean) / std
 # transform face into one sample
 samples = expand_dims(face_pixels, axis=0)
 # make prediction to get embedding
 yhat = model.embeddings(samples)
 return yhat[0]

# load the face dataset
data = load('.\\face-detection.npz')
trainX, trainy  = data['arr_0'], data['arr_1']
print('Loaded: ', trainX.shape, trainy.shape, )
# load the facenet model
print('Loading Model')
MyFaceNet = FaceNet()
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
 embedding = get_embedding(MyFaceNet, face_pixels)
 newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# save arrays to one file in compressed format
savez_compressed('.\\embeddings\\embeddings.npz', newTrainX, trainy)

Loaded:  (171, 160, 160, 3) (171,)
Loading Model

Loaded Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 

Face Identification program

In [3]:
import cv2
import numpy as np
from mtcnn import MTCNN
from tensorflow.keras.models import load_model
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [10]:
# Load face detection model
detector = MTCNN()

# Load FaceNet model
facenet_model = FaceNet()

# Load face embeddings
data = np.load('.\\embeddings\\embeddings.npz')
trainX, trainy = data['arr_0'], data['arr_1']

# Normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)

# Label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)

# Define the classes
class_names = out_encoder.classes_
class_names = np.append(class_names, 'unknown')

# Train SVM classifier
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)

# Define function to extract face embeddings
def extract_face_embeddings(image):
    # Detect faces in the image
    faces = detector.detect_faces(image)
    if not faces:
        return None
    # Extract the first face only
    x1, y1, width, height = faces[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    # Resize face to the size required by facenet model
    face = cv2.resize(face, (160, 160))
    # Preprocess the face for facenet model
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    face = np.expand_dims(face, axis=0)
    # Generate embeddings using facenet model
    embeddings = facenet_model.embeddings(face)
    return embeddings[0]

# Define function to identify the identity of an input image
def identify_person(image):
    # Extract face embeddings from input image
    embeddings = extract_face_embeddings(image)
    if embeddings is None:
        return None, None
    # Normalize embeddings
    embeddings = in_encoder.transform([embeddings])
    # Predict the identity and confidence using SVM classifier
    prediction = model.predict(embeddings)
    confidence = model.predict_proba(embeddings)[0][prediction] * 100
    prediction = out_encoder.inverse_transform(prediction)
    return prediction[0].item(), confidence

# Define function to identify the identity and confidence of an input image
def identify_person_with_unknown(image, threshold=0.9):
    # Extract face embeddings from input image
    embeddings = extract_face_embeddings(image)
    if embeddings is None:
        return None, None
    # Normalize embeddings
    embeddings = in_encoder.transform([embeddings])
    # Predict the identity and confidence using SVM classifier
    predictions = model.predict_proba(embeddings)[0]
    max_idx = np.argmax(predictions)
    if predictions[max_idx] >= threshold:
        prediction = out_encoder.inverse_transform([max_idx])
        confidence = predictions[max_idx] * 100
        return prediction[0].item(), confidence
    else:
        return "unknown", None

# Example usage
image = cv2.imread('.\\test\\aswin\\s3_41.png')
person, confidence = identify_person_with_unknown(image)
if person is None:
    print('No face detected in the input image!')
elif person == "unknown":
    text = "Unknown person"
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(text)
    plt.axis('off')
    plt.show()
else:
    # Display the predicted name and confidence probability
    text = f'Predicted: {str(person)} ({confidence:.2f}%)'
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(text)
    plt.axis('off')
    plt.show()


No face detected in the input image!
